In [36]:
import dash
from dash import dcc, html, Input, Output
import datetime as dt
import pandas as pd
import plotly.express as px

# Load dataset
covid_data = pd.read_csv(r'C:\Users\khale\OneDrive\Desktop\CLS\Assignments\DashBoard\Covid_DataSet.csv')

# Convert 'Date' column to datetime format
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

# Initialize the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
server = app.server

# **Styles for background and fonts**

In [37]:
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        <title>COVID-19 Statistics</title>
        <style>
            body {
                background-image: url('https://thedaily.case.edu/wp-content/uploads/2023/12/covid-19.jpg');
                background-size: cover;
                font-family: 'Arial, sans-serif';
                line-height: 1.6;
                margin: 0;
                padding: 0;
            }
            .container {
                background: rgba(255, 255, 255, 0.9);
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
                max-width: 1200px;
                margin: 20px auto;
                text-align: justify;
            }
            h1 {
                text-align: center;
                font-size: 2.5em;
                margin-bottom: 20px;
                color: #333;
            }
            .footer {
                text-align: center;
                margin-top: 20px;
                font-size: 1.2em;
                font-weight: bold;
                color: #555;
            }
        </style>
    </head>
    <body>
        <div class="container">
            {%app_entry%}
        </div>
        <footer class="footer">
            Better Call Karam
        </footer>
        {%config%}
        {%scripts%}
        {%renderer%}
    </body>
</html>
'''

# **Layout of the dashboard**

In [42]:
app.layout = html.Div([
    html.H1("COVID-19 Statistics"),

    # Dropdown to select country
    html.Label("Select Country:"),
    dcc.Dropdown(
        id="country-dropdown",
        options=[
            {"label": country, "value": country} for country in covid_data['Country/Region'].unique()
        ],
        value="Algeria",
        placeholder="Select a country",
    ),

    html.Br(),

    # Display WHO region
    html.Div(id="who-region"),

    html.Br(),

    # Date picker for selecting date range
    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
    id="date-picker-range",
    min_date_allowed=dt.datetime(2020, 1, 22),
    max_date_allowed=dt.datetime(2020, 7, 27),
    start_date=dt.datetime(2020, 1, 22),
    end_date=dt.datetime(2020, 7, 27),
    display_format="DD/MM/YYYY",  # Ensure the format matches your data
    style={"margin-bottom": "20px"}
    ),

    html.Br(),
    html.Br(),

    # Graphs for COVID-19 statistics
    dcc.Graph(id="covid-graph")
])

# **The Callback to update WHO region and graph**

In [39]:
@app.callback(
    [Output("who-region", "children"),
    Output("covid-graph", "figure")],
    [Input("country-dropdown", "value"),
    Input("date-picker-range", "start_date"),
    Input("date-picker-range", "end_date")]
)
def update_dashboard(selected_country, start_date, end_date):
    if not selected_country:
        return "Please select a country.", {}

    # Filter data based on selected country and date range
    filtered_data = covid_data[(covid_data['Country/Region'] == selected_country) &
                                (covid_data['Date'] >= start_date) &
                                (covid_data['Date'] <= end_date)]

    # Get WHO region for the selected country
    who_region = filtered_data['WHO Region'].iloc[0] if not filtered_data.empty else "N/A"

    # Create line chart
    fig = px.line(
        filtered_data,
        x="Date",
        y=["Confirmed", "Active", "Recovered", "Deaths"],
        labels={"value": "Cases", "variable": "Category"},
        title=f"COVID-19 Statistics for {selected_country}"
    )

    return f"WHO Region: {who_region}", fig

# **Run the app**

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=True, port=8052)


# **Vedio Link**

https://drive.google.com/file/d/1jH7qOmHtMczTkcqdLFQ3Yggu5hVJnTX1/view?usp=sharing